In [1]:
! pip install transformers
! pip install jiwer

# Importing all necessary files

In [2]:
from transformers import pipeline
from transformers import BertForQuestionAnswering, BertTokenizer
from transformers import RobertaForQuestionAnswering, RobertaTokenizer
from transformers import XLNetTokenizer, TFXLNetForQuestionAnsweringSimple
import pandas as pd
import torch
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import f1_score
from sklearn.feature_extraction.text import CountVectorizer
import csv
from jiwer import wer

# Converting CSV into list of dictionaries

In [3]:
data = pd.read_csv('/content/Q&A.csv')
data.head(3)

,Context,Question,Answer
0,"Architecturally, the school has a Catholic cha...",To whom did the Virgin Mary allegedly appear i...,Saint Bernadette Soubirous
1,"As at most other universities, Notre Dame's st...",When did the Scholastic Magazine of Notre dame...,September 1876
2,The university is the major seat of the Congre...,Where is the headquarters of the Congregation ...,Rome


In [4]:
with open('/content/Q&A.csv') as f:
    data_dictionary = [{k: v for k, v in row.items()}
                 for row in csv.DictReader(f, skipinitialspace=True)]
    print(len(data_dictionary))
    print(data_dictionary[0])
    


20
{'Context': 'Architecturally, the school has a Catholic character. Atop the Main Building\'s gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend \\"Venite Ad Me Omnes\\". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.', 'Question': 'To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?', 'Answer': 'Saint Bernadette Soubirous'}


# Model Initialization

In [6]:
def initialize_model(model_name):

  ''' 
      input : name of the model
      output : model, tokenizer
  '''


  if model_name == 'bert-large-uncased-whole-word-masking-finetuned-squad':
    model = BertForQuestionAnswering.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')
    tokenizer = BertTokenizer.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')
  elif model_name == 'roberta-base':
    model = RobertaForQuestionAnswering.from_pretrained('roberta-base')
    tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
  return model, tokenizer


# Model result

In [7]:
def get_model_result(data_dictionary, model, tokenizer, model_name):

  '''
       input : data_dictionary, model which we are using, tokenizer of particular model,
               & name of the model
       output: dictionary 
         
  '''

  for i in range(len(data_dictionary)):
    question = data_dictionary[i]['Question']
    context = data_dictionary[i]['Context']
    question_answering = pipeline('question-answering', model= model, tokenizer=tokenizer)
    result = question_answering(question=question, context=context)
    data_dictionary[i][model_name + 'predictions'] = result['answer']
      
  return data_dictionary

# Consine prediction

In [8]:
def cosine(answer, prediction):
  '''
  input : actual answer, and predicted answer
  output: cosine similarity value

  '''
  cv = CountVectorizer(max_features=500)
  for i in range(len(data_dictionary)):
    answers = [answer, prediction]
    vectors = cv.fit_transform(answers).toarray()
    similarity = cosine_similarity(vectors)
  return similarity[1][0]




# WER prediction

In [9]:
def word_error_rate(answer, prediction):

  '''
  input : actual answer, predicted answer
  output : 
  
  '''
  for i in range(len(data_dictionary)):
    similarity = 1 - wer(answer, prediction)
  return similarity


# F1 Score prediction

In [10]:
def f1_score(answer, prediction):

  '''
    input: actuall answer, predicted answer'
    output: F1 Score
  '''

  for i in range(len(data_dictionary)):
    answer_tokens = answer.split()
    prediction_tokens = prediction.split()
    if len(prediction_tokens) == 0 or len(answer_tokens):
      fi_pred = (int(prediction_tokens == answer_tokens))
    
    common_tokens = set(prediction_tokens) & set(answer_tokens)
    if len(common_tokens) == 0:
      fi_pred = 0 
    
    else:
      precision = len(common_tokens) / len(prediction_tokens)
      recall = len(common_tokens) / len(answer_tokens)
      fi_pred = (2 * (precision * recall) / (precision + recall))
  return fi_pred


# Metric Evaluation

In [11]:
def get_evaluation_results(data_dictionary, metric_name, model_name):

  '''
      input: dictionary, name of metric & model name
      output: dictionary
  '''
  for item in data_dictionary:
    if metric_name == 'cosine':
      item[model_name+' cosine similarity'] = cosine(item['Answer'], item[model_name +'predictions'])
    elif metric_name == 'wer':
      item[model_name+' wer'] = word_error_rate(item['Answer'], item[model_name +'predictions'])
    elif metric_name == 'F1Score':
      item[model_name+' F1Score'] = f1_score(item['Answer'], item[model_name+'predictions'])
  return data_dictionary

# Final function


In [12]:
def question_answering(model_names,data_dictionary,evaluation_metrics_name,evaluation=True):
  '''
  input : name of the model, dictionary of data, metric name 
  output : required dictionary
  '''
  for model_name in model_names:
    model,tokenizer=initialize_model('bert-large-uncased-whole-word-masking-finetuned-squad')
    data_dictionary=get_model_result(data_dictionary,model, tokenizer,model_name)
    if evaluation:
      for metric in evaluation_metrics_name:
        data_dictionary = get_evaluation_results(data_dictionary,metric,model_name)
  return data_dictionary

In [13]:
model_names = ['bert-large-uncased-whole-word-masking-finetuned-squad']
data_dictionary = data_dictionary
evaluation_metrics_name = ['cosine']

In [14]:
question_answering(model_names,data_dictionary,evaluation_metrics_name,evaluation=True)

[{'Answer': 'Saint Bernadette Soubirous',
  'Context': 'Architecturally, the school has a Catholic character. Atop the Main Building\'s gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend \\"Venite Ad Me Omnes\\". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.',
  'Question': 'To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?',
  'bert-large-uncased-whole-word-masking-finetuned-squad cosine similarity': 1.0000000000000002,
  'bert-large-uncased-whole-word-masking-finetuned-squa